In [3]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [4]:
# Load environment variables
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    print("No API key found")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but doesn't start with 'sk-proj-")
elif api_key.strip() != api_key:
    print("An API key was found but looks like there might be space or special characters in the end")
else:
    print("API key found and looks good so far")

API key found and looks good so far


In [5]:
openai = OpenAI()

# Preview

In [7]:
message = "Hello GPT! How is the weather today?"
response = openai.chat.completions.create(model="gpt-4o-mini", 
                                          messages = [{"role": "user",
                                                     "content":message}])
print(response.choices[0].message.content) 

Hello! I'm unable to check real-time weather updates since I don't have access to live data. However, you can easily find the current weather by checking a weather website, app, or asking a smart device. If you need help with anything else, feel free to ask!


# Web Parsing 

In [8]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [10]:
class Website:

    def __init__(self, url):
        """
        Create a Website object from the given url using the Beautiful Soup Library
        """
        self.url = url 
        response = requests.get(url=url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        self.title = soup.title.string if soup.title else "No Title"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [11]:
wiki = Website("https://en.wikipedia.org/wiki/Robert_Frost")
print(wiki.title)
print(wiki.text)

Robert Frost - Wikipedia
Jump to content
Main menu
Main menu
move to sidebar
hide
Navigation
Main page
Contents
Current events
Random article
About Wikipedia
Contact us
Contribute
Help
Learn to edit
Community portal
Recent changes
Upload file
Search
Search
Appearance
Donate
Create account
Log in
Personal tools
Donate
Create account
Log in
Pages for logged out editors
learn more
Contributions
Talk
Contents
move to sidebar
hide
(Top)
1
Biography
Toggle Biography subsection
1.1
Early life
1.2
Adult years
1.3
Personal life
2
Work
Toggle Work subsection
2.1
Style and critical reception
2.2
Themes
2.3
Influenced by
2.4
Influenced
3
Awards and recognition
Toggle Awards and recognition subsection
3.1
Pulitzer Prizes
4
Legacy and cultural influence
5
Selected works
Toggle Selected works subsection
5.1
Poetry collections
5.2
Plays
5.3
Letters
5.4
Other
6
See also
7
Citations
8
General sources
9
External links
Toggle External links subsection
9.1
Libraries
9.2
Electronic editions
Toggle the table

## Define prompts

In [12]:
system_prompt = """
                    You are an assistant that analyzes the content of a website and provides a short summary,
                    ignoring text that might be related to navigation. Response should be in markdown
                """

In [13]:
def user_prompt(website):
    user_prompt = f"You are looking at a website titled (website.title)"
    user_prompt += "\n The contents of this website are as follows; \
                    please provide a short summary of this website in markdown. "
    user_prompt += website.text
    return user_prompt

In [14]:
print(user_prompt(wiki))

You are looking at a website titled (website.title)
 The contents of this website are as follows;                     please provide a short summary of this website in markdown. Jump to content
Main menu
Main menu
move to sidebar
hide
Navigation
Main page
Contents
Current events
Random article
About Wikipedia
Contact us
Contribute
Help
Learn to edit
Community portal
Recent changes
Upload file
Search
Search
Appearance
Donate
Create account
Log in
Personal tools
Donate
Create account
Log in
Pages for logged out editors
learn more
Contributions
Talk
Contents
move to sidebar
hide
(Top)
1
Biography
Toggle Biography subsection
1.1
Early life
1.2
Adult years
1.3
Personal life
2
Work
Toggle Work subsection
2.1
Style and critical reception
2.2
Themes
2.3
Influenced by
2.4
Influenced
3
Awards and recognition
Toggle Awards and recognition subsection
3.1
Pulitzer Prizes
4
Legacy and cultural influence
5
Selected works
Toggle Selected works subsection
5.1
Poetry collections
5.2
Plays
5.3
Letters
5.

In [16]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2+2"},
]

In [17]:
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

Oh, a tough one! Let me put on my thinking cap... It's 4. Shocking, right?


In [18]:
def create_messages(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt(website)},
    ]

In [21]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = create_messages(website)
    )
    return response.choices[0].message.content

In [22]:
summarize("https://en.wikipedia.org/wiki/Robert_Frost")

'# Robert Frost - Summary\n\n**Robert Frost** (1874–1963) was a prominent American poet renowned for his realistic depictions of rural life and mastery of colloquial speech. He frequently drew inspiration from the settings of New England, using these experiences to explore intricate social and philosophical themes. Frost\'s literary career was marked by significant acclaim, earning him four Pulitzer Prizes for Poetry and establishing him as a leading figure in American literature, often referred to as a "public literary figure" or an "artistic institution."\n\n## Biography\nFrost was born in San Francisco and faced considerable personal tragedy throughout his life, including the loss of his father at a young age and struggles with depression in his family. He attended various educational institutions but left Harvard due to health issues. Many of his early poems were written while running a farm in New Hampshire, and his first significant works were published in England before his retu

In [23]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [24]:
display_summary("https://en.wikipedia.org/wiki/Robert_Frost")

# Robert Frost - Summary

Robert Lee Frost (1874–1963) was an influential American poet renowned for his poignant depictions of rural life, particularly in New England, with a focus on complex social and philosophical themes. Over his lifetime, Frost received widespread recognition, including four Pulitzer Prizes for Poetry, and he is celebrated as one of America's prominent literary figures.

## Early Life
Frost was born in San Francisco, California, and faced significant hardships in childhood, including the death of his father when he was 11. His family's financial situation forced them to move to Massachusetts. He showed an early talent for writing and graduated from high school in 1892. Frost briefly attended Dartmouth College and Harvard University but left due to illness.

## Personal Life
Frost married Elinor Miriam White in 1895, and they had six children, but many faced tragic fates, including two who died young. His later life was marked by personal losses and struggles with depression, which affected both him and his family.

## Literary Career
Frost's literary career took off after moving to England in 1912, where he published his first collection, *A Boy's Will* (1913), followed by *North of Boston* (1914). His poetry often utilized traditional forms while embodying Modernist elements, reflecting themes of nature, humanity, and existential questions. His notable works include "The Road Not Taken," "Stopping by Woods on a Snowy Evening," and "Fire and Ice."

Frost was also a prolific educator, teaching at several notable institutions, including Dartmouth and Amherst College, and he became known for his conversational poetry style, which captured the rhythms of everyday speech.

## Awards and Legacy
Frost was awarded honorary degrees from numerous universities and held the title of Poet Laureate of Vermont. He was nominated for the Nobel Prize in Literature 31 times. His work continues to resonate, influencing generations of poets and garnering a place in popular culture, referenced in various media and recognized in scholarly discussions on American literature. Frost died in 1963, leaving behind a lasting legacy as one of America's greatest poets.